In [1]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
load_dotenv(dotenv_path='.env', override=True)

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# https://console.groq.com/keys
groq_api_key = os.getenv("GROQ_API_KEY")

# https://console.anthropic.com/dashboard
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# https://aistudio.google.com/app/apikey
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [4]:
# pip install tabulate

from sqlalchemy import create_engine, inspect
from tabulate import tabulate

base = 'sqlite:///loja.db'

def get_schema(base,tabela):
    engine = create_engine(base)
    inspector = inspect(engine)
    columns = inspector.get_columns(tabela)

    column_data = [
        {
            "Column Name": col["name"],
            "Data Type": str(col["type"]),
            "Nullable": "Yes" if col["nullable"] else "No",
            "Default": col["default"] if col["default"] else "None",
            #"Autoincrement": "Yes" if col["autoincrement"] else "No",
        }
        for col in columns
    ]
    schema_output = tabulate(column_data, headers="keys", tablefmt="grid")
    formatted_schema = f"Schema for 'PRODUCTS' table:\n{schema_output}"

    return formatted_schema

print(get_schema(base,"products"))

Schema for 'PRODUCTS' table:
+---------------+-------------+------------+-----------+
| Column Name   | Data Type   | Nullable   | Default   |
+===============+=============+============+===========+
| id            | INTEGER     | Yes        | None      |
+---------------+-------------+------------+-----------+
| name          | TEXT        | No         | None      |
+---------------+-------------+------------+-----------+
| color         | TEXT        | No         | None      |
+---------------+-------------+------------+-----------+
| price         | REAL        | No         | None      |
+---------------+-------------+------------+-----------+


In [5]:
from langchain_community.utilities import SQLDatabase
from pathlib import Path

db_string = f"sqlite:///loja.db"
db = SQLDatabase.from_uri(db_string, sample_rows_in_table_info=0)

def get_schema(_):
    schemas = ''
    for table in db.get_usable_table_names():
        schemas += db.get_table_info([table]).split('/*')[0].strip()

    return schemas

print(get_schema(_))

CREATE TABLE products (
	id INTEGER, 
	name TEXT NOT NULL, 
	color TEXT NOT NULL, 
	price REAL NOT NULL, 
	PRIMARY KEY (id)
)


In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini", max_tokens=256, openai_api_key=openai_api_key)
user_question= "I need the top 2 products where the price is less than 2.00. Answer in portuguese"
messages = [{"role": "user", "content": user_question}]
response = llm.invoke(messages)
print(response.content)

Claro! Aqui estão os dois principais produtos com preço inferior a 2,00:

1. Produto A - R$ 1,50
2. Produto B - R$ 1,80

Se precisar de mais informações, é só avisar!


In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

template = """
Baseado no schema da tabela abaixo, escreva um comando SQL que responda a pergunta do usuário. Apresente como resposta apenas o comando SQL.
Não coloque ```sql na resposta
{schema}
Pergunta: {question}
SQL: 
"""

prompt = ChatPromptTemplate.from_template(template)

sql_chain = (
    RunnablePassthrough.assign(
      schema=lambda _: get_schema(_)
    )
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

user_query = "Quais produtos tem preço menor que 2.00?"
result = sql_chain.invoke({
    "question": user_query
})
print(result)


SELECT * FROM products WHERE price < 2.00;


In [9]:
template = """
Baseado no schema da tabela abaixo, escreva um comando SQL que responda a pergunta do usuário. Apresente a resposta em linguagem natural.
{schema}
Pergunta: {question}
SQL: {query}
Resposta: {response}
"""

prompt_response = ChatPromptTemplate.from_template(template)

def run_query(query):
    return db.run(query)

full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
      schema=get_schema,
      response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | llm
    | StrOutputParser()
)

user_query = "Quais produtos tem preço menor que 2.00?"
result = full_chain.invoke({
    "question": user_query
})
print(result)


Com base na consulta realizada, os produtos que têm preço menor que 2,00 são:

1. Caneta marrom - R$ 1,00
2. Caneta preta - R$ 1,50
3. Caneta marrom - R$ 1,00
4. Caneta preta - R$ 1,50
5. Caneta marrom - R$ 1,00
6. Caneta preta - R$ 1,50

Esses produtos estão disponíveis a preços acessíveis, todos abaixo de R$ 2,00.
